In [6]:
# importing libraries
import cv2
import os
from os import listdir

In [7]:
# function to convert video to images
def video_to_images(video, folder_name):

    if os.path.exists(folder_name):
        for file_name in listdir(folder_name):
            string1 = ""
            string1 = str(folder_name) + "/" + str(file_name)
            os.remove(string1)
    else:
        os.makedirs(folder_name)

    while(1):
        # extracting the frames
        ret, frame = video.read()
        # if frame is extracted
        if ret == True:
            # writing the extracted frame
            cv2.imwrite('./'+folder_name+'/'+str(video.get(1))+'.jpg', frame)
        else:
            break

    # releasing the video object
    video.release()
    # closing all the windows
    cv2.destroyAllWindows()

In [8]:
# function to convert images to video
def images_to_video(image_folder, video_name, fps):
    
    images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
    images.sort(key=lambda x: int(x.split('.')[0]))

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))


    cv2.destroyAllWindows()
    video.release()

In [9]:
# declaring the videos
video_to_be_composited = cv2.VideoCapture('./yt1s.com - Chipi Chipi Chapa Chapa CAT GREEN SCREEN.mp4') # foreground video
video_to_be_composited_on = cv2.VideoCapture('./yt1s.com - Free Halloween Graveyard Cemetery Stock Video Footage.mp4') # background video

In [10]:
# converting to images
video_to_images(video_to_be_composited, 'video_to_be_composited')
video_to_images(video_to_be_composited_on, 'video_to_be_composited_on')

In [11]:
# function to composite the frames
def composite_videos( output_folder, video1frames, video2frames, final_height, final_width):

    counter = 0
    no_of_frames = min(len(os.listdir(video1frames)) , len(os.listdir(video2frames)))

    image_folder1 = video1frames
    image_folder2 = video2frames

    images1 = [img for img in os.listdir(image_folder1) if img.endswith(".jpg")]
    images2 = [img for img in os.listdir(image_folder2) if img.endswith(".jpg")]

    images1.sort(key=lambda x: int(x.split('.')[0]))
    images2.sort(key=lambda x: int(x.split('.')[0]))

    if os.path.exists(output_folder):
        os.system('rm -rf ' + output_folder)
    os.system('mkdir ' + output_folder)

    if os.path.exists(output_folder):
        for file_name in listdir(output_folder):
            string1 = ""
            string1 = str(output_folder) + "/" + str(file_name)
            os.remove(string1)
    else:
        os.makedirs(output_folder)

    while True:
            
            if counter >= no_of_frames:
                break
    
            imgtemp =  cv2.imread(image_folder1 + images1[counter])
            imgbg = cv2.imread(image_folder2 + images2[counter])
    
            # setting the output frame size
            imgbg = cv2.resize(imgbg, (final_width, final_height))
            imgtemp = cv2.resize(imgtemp, (imgbg.shape[1], imgbg.shape[0]))
    
            for i in range(imgtemp.shape[0]):
                for j in range(imgtemp.shape[1]):
                    if imgtemp[i][j][1] >= 200: # checking if green pixel
                        imgtemp[i][j]= imgbg[i][j]

                        
            # writing the video
            cv2.imwrite(output_folder+'/'+str(counter)+'.jpg', imgtemp)
            counter += 1   
            print(str(counter) + ' / ' + str(no_of_frames) , end='\r')
    

In [12]:
final_height = 480
final_width = 640
output_folder = 'output'
frames_folder1 = 'video_to_be_composited/' # foreground video (green screen)
frames_folder2 = 'video_to_be_composited_on/' # background video
composite_videos(output_folder, frames_folder1, frames_folder2, final_height, final_width)

In [13]:
os.remove('./output.mp4')
images_to_video('./output', 'output.mp4', 30)